In [ ]:
# Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from skimage.util import random_noise
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from tensorflow.keras import backend as K

In [ ]:
# Directories

In [ ]:
os.makedirs("models", exist_ok=True)
os.makedirs("results", exist_ok=True)

In [ ]:
(x_train, _), (x_test, _) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [ ]:
# Visualize sample clean images

In [ ]:
plt.figure(figsize=(10,2))
for i in range(10):
    plt.subplot(1,10,i+1)
    plt.imshow(x_train[i])
    plt.axis('off')
plt.suptitle("Sample CIFAR-10 Images (Clean)")
plt.show()

In [ ]:
# Add Noise

In [ ]:
def add_noise(data, mode='gaussian', var=0.01, amount=0.02):
    if mode == 'gaussian':
        noisy = random_noise(data, mode='gaussian', var=var)
    elif mode == 's&p':
        noisy = random_noise(data, mode='s&p', amount=amount)
    else:
        noisy = data
    noisy = np.clip(noisy, 0., 1.)
    return noisy

x_train_noisy = add_noise(x_train, 'gaussian', 0.01)
x_test_noisy = add_noise(x_test, 'gaussian', 0.01)

In [ ]:
# Visualize noisy vs clean

In [ ]:
plt.figure(figsize=(20,4))
for i in range(10):
    # Noisy
    ax = plt.subplot(2,10,i+1)
    plt.imshow(x_train_noisy[i])
    plt.axis('off')
    if i==0: plt.ylabel("Noisy")
    # Original
    ax = plt.subplot(2,10,i+11)
    plt.imshow(x_train[i])
    plt.axis('off')
    if i==0: plt.ylabel("Clean")
plt.show()